SCRIPT PARA CARGAR EL MODELO ENTRENADO EXPORTADO Y EL DATASET A UTILIZAR
PARA LAS RECOMENDACIONES

In [1]:
# Importamos la librería Pickle para cargar el modelo entrenado y guardado
import pickle

# Cargamos el modelo
svd = pickle.load(open("model.pkl","rb"))

In [4]:
# Importamos la libreria Difflib para la bùsqueda de strings y random para ofrecer
# recomendaciones al azar
import difflib
import random


# Creamos las funciones a utilizar
def get_restaurant_id(restaurant_title, metadata):
    '''
    Busca el gmap_id del restaurant cuyo nombre se pasa como parámetro en el 
    dataset y devuelve su gmap_id
    '''    
    existing_titles = list(metadata['name'].values)
    closest_titles = difflib.get_close_matches(restaurant_title, existing_titles)
    restaurant_id = metadata[metadata['name'] == closest_titles[0]]['gmap_id'].values[0]
    return restaurant_id

def get_restaurant_info(restaurant_id, metadata):
    '''
    Accede a los datos del restaurant seleccionado y genera un diccionario con los
    mismos para configurar la salida de la función principal
    '''   
    restaurant_info = metadata[metadata['gmap_id'] == restaurant_id][['gmap_id', 'name', 
                                                    'address', 'avg_rating', 'description', 'description_x', 'city']]
    return restaurant_info.to_dict(orient='records')

def predict_review(user_id, restaurant_title, model, metadata):
    '''
    Predice y retorna el puntaje que el usuario pasado como parámetro le daría al restaurante
    pasado como parámetro
    '''   
    restaurant_id = get_restaurant_id(restaurant_title, metadata)
    review_prediction = model.predict(uid=user_id, iid=restaurant_id)
    return review_prediction.est

def generate_recommendation(user_id, model, metadata, thresh=4.0):
    '''
    Genera una recomendación aleatoria de restaurantes para el usuario indicado
    sobre todas las opciones del dataset, tomando como única condición
    que el puntaje que el usuario le otorgaría al mismo supere el threshold
    pasado como parámetro
    '''   
    restaurant_titles = list(metadata['name'].values)
    random.shuffle(restaurant_titles)
    
    for restaurant_title in restaurant_titles:
        rating = predict_review(user_id, restaurant_title, model, metadata)
        if rating >= thresh:
            restaurant_id = get_restaurant_id(restaurant_title, metadata)
            return get_restaurant_info(restaurant_id, metadata)
            
def generate_recommendation_by_city(user_id, city, model, metadata, thresh=4.0):
    '''
    Genera una recomendación aleatoria de restaurantes para el usuario indicado
    sobre la ciudad indicada, tomando como condición también que el puntaje que
    el usuario le otorgaría al mismo supere el threshold pasado como parámetro
    '''   
    restaurant_city = metadata[metadata["city"] == city]
    restaurant_titles = list(restaurant_city['name'].values)
    random.shuffle(restaurant_titles)
    
    for restaurant_title in restaurant_titles:
        rating = predict_review(user_id, restaurant_title, model, metadata)
        if rating >= thresh:
            restaurant_id = get_restaurant_id(restaurant_title, metadata)
            return get_restaurant_info(restaurant_id, metadata)

In [2]:
# Importamos Pandas para importar el dataser con los restaurantes de EEUU
import pandas as pd
comercios_comida = pd.read_csv("./comercios_comida.csv")

In [5]:
# Generamos una recomendación de ejemplo para el usuario cuyo ID es
# 113055753770388899087 en la ciudad de Cleveland
generate_recommendation_by_city(113055753770388899087,  "Cleveland", svd, comercios_comida)

[{'gmap_id': '0x8830efadb77d6549:0x645997dd6f76a63d',
  'name': 'Brooklyn Pub',
  'address': 'Brooklyn Pub, 6900 Biddulph Rd, Cleveland, OH 44144',
  'avg_rating': 4.3,
  'description': nan,
  'description_x': 'Bar',
  'city': 'Cleveland'}]